In [ ]:
pip install spacy

In [3]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=f33f672b6a8d0decc6f9cff393a51c2c98a0a155c001050dd87dd61b319a18a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-1sk4mqny/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [1]:
import spacy
nlp = spacy.load("en_core_web_md")

In [3]:
from spacy import displacy

## 2. Come up with 3 patterns for this relation
*Use visualizations of the dependency structure to help find the patterns!*

In [15]:
text0 = "Sony Corporation was founded by Masaru Ibuka, Akio Morita in 7 May 1946 as one of the leading manufacturers of electronic products for the consumer and professional markets, and a leading player in the film and television entertainment industry."

text1 = "LG Corporation was founded by Koo In-hwoi in 5 January 1947 as Lak Hui Chemical Industrial Corp."

text2 = "Panasonic Corporation was founded by Kōnosuke Matsushita in March 13, 1918 as a vendor of duplex lamp sockets."

doc0 = nlp(text0)
doc1 = nlp(text1)
doc2 = nlp(text2)

In [16]:
docs = (doc0, doc1, doc2)
for i, doc in enumerate(docs):
    print(f"doc {i}".center(100, "-"))
    displacy.render(doc, style='dep', jupyter=True)
    for token in doc:
        print(f"text: {token.text}, pos: {token.pos_}, dep: {token.dep_}, head: {token.head.text}")

-----------------------------------------------doc 0------------------------------------------------


text: Sony, pos: PROPN, dep: compound, head: Corporation
text: Corporation, pos: PROPN, dep: nsubjpass, head: founded
text: was, pos: AUX, dep: auxpass, head: founded
text: founded, pos: VERB, dep: ROOT, head: founded
text: by, pos: ADP, dep: agent, head: founded
text: Masaru, pos: PROPN, dep: compound, head: Ibuka
text: Ibuka, pos: PROPN, dep: pobj, head: by
text: ,, pos: PUNCT, dep: punct, head: Ibuka
text: Akio, pos: PROPN, dep: compound, head: Morita
text: Morita, pos: PROPN, dep: appos, head: Ibuka
text: in, pos: ADP, dep: prep, head: Morita
text: 7, pos: NUM, dep: nummod, head: May
text: May, pos: PROPN, dep: pobj, head: in
text: 1946, pos: NUM, dep: nummod, head: May
text: as, pos: SCONJ, dep: prep, head: founded
text: one, pos: NUM, dep: pobj, head: as
text: of, pos: ADP, dep: prep, head: one
text: the, pos: DET, dep: det, head: manufacturers
text: leading, pos: VERB, dep: amod, head: manufacturers
text: manufacturers, pos: NOUN, dep: pobj, head: of
text: of, pos: ADP, dep: pre

text: LG, pos: PROPN, dep: compound, head: Corporation
text: Corporation, pos: PROPN, dep: nsubjpass, head: founded
text: was, pos: AUX, dep: auxpass, head: founded
text: founded, pos: VERB, dep: ROOT, head: founded
text: by, pos: ADP, dep: agent, head: founded
text: Koo, pos: PROPN, dep: compound, head: hwoi
text: In, pos: PROPN, dep: compound, head: hwoi
text: -, pos: PUNCT, dep: punct, head: hwoi
text: hwoi, pos: PROPN, dep: pobj, head: by
text: in, pos: ADP, dep: prep, head: founded
text: 5, pos: NUM, dep: nummod, head: January
text: January, pos: PROPN, dep: pobj, head: in
text: 1947, pos: NUM, dep: nummod, head: January
text: as, pos: SCONJ, dep: prep, head: founded
text: Lak, pos: PROPN, dep: compound, head: Hui
text: Hui, pos: PROPN, dep: compound, head: Chemical
text: Chemical, pos: PROPN, dep: compound, head: Industrial
text: Industrial, pos: PROPN, dep: compound, head: Corp.
text: Corp., pos: PROPN, dep: pobj, head: as
-----------------------------------------------doc 2----

text: Panasonic, pos: PROPN, dep: compound, head: Corporation
text: Corporation, pos: PROPN, dep: nsubjpass, head: founded
text: was, pos: AUX, dep: auxpass, head: founded
text: founded, pos: VERB, dep: ROOT, head: founded
text: by, pos: ADP, dep: agent, head: founded
text: Kōnosuke, pos: PROPN, dep: compound, head: Matsushita
text: Matsushita, pos: PROPN, dep: pobj, head: by
text: in, pos: ADP, dep: prep, head: founded
text: March, pos: PROPN, dep: pobj, head: in
text: 13, pos: NUM, dep: nummod, head: March
text: ,, pos: PUNCT, dep: punct, head: March
text: 1918, pos: NUM, dep: nummod, head: March
text: as, pos: SCONJ, dep: prep, head: founded
text: a, pos: DET, dep: det, head: vendor
text: vendor, pos: NOUN, dep: pobj, head: as
text: of, pos: ADP, dep: prep, head: vendor
text: duplex, pos: NOUN, dep: compound, head: lamp
text: lamp, pos: NOUN, dep: compound, head: sockets
text: sockets, pos: NOUN, dep: pobj, head: of
text: ., pos: PUNCT, dep: punct, head: founded


According to these pictures, trivial situation for "org_name was founded by founder in date" is:

* To get org name:  founded [ROOT] -> nsubjpass -> org_name -> other_parts_of_name[compound]
* To get a founder: founded [ROOT] -> by [agent] -> part_of_name [pobj] -> other_parts_of_name[compound]
* To get a date: founded [ROOT] -> in [prep] -> date [pobj] -> [any other dependencies]

To get info if a founder name mentioned non-straight like "According to the magazine, founder X, founder of org_name":

* To get org_name: founder [appos] <- founder <- part_of_name [compound] 

## 4. Use the spaCy Matcher to create the patterns.

In [17]:
from spacy.matcher import DependencyMatcher
from copy import deepcopy

def process_matches(pattern, doc):
    matcher = DependencyMatcher(doc.vocab)
    matcher.add("pattern", None, pattern)
    matches = matcher(doc)
    _, matches = matches[0]
    return matches


def dependency_parser(pattern, doc, additional_pattern=None):
    matches = None
    if additional_pattern:
        compounded_pattern = deepcopy(pattern)
        compounded_pattern.append(additional_pattern)
        matches = process_matches(compounded_pattern, doc)
    if not matches:
        matches = process_matches(pattern, doc)
    return [[doc[pos] for pos in m]
           for m in matches]

# additional pattern for names
compound_pattern = {
    "PATTERN": {"DEP": "compound", "OP": "+"},
    "SPEC": {"NODE_NAME": "val_compounds", 'NBOR_NAME': "val", "NBOR_RELOP": ">"}
}

# First we want to create a bunch of patternt to define an organization which was founded
founded_pattern = [
    {
    "PATTERN": {"LEMMA": "found"},
    "SPEC": {"NODE_NAME": "root"}
    },
    {
    "PATTERN": {"DEP": "nsubjpass"},
    "SPEC": {"NODE_NAME": "val", 'NBOR_NAME': "root", "NBOR_RELOP": ">"}
    }
]

# Next we want to determine a founder
founder_pattern = [
    {
    "PATTERN": {"LEMMA": "found"},
    "SPEC": {"NODE_NAME": "root"}
    },
    {
    "PATTERN": {"DEP": "agent", "LEMMA": "by"},
    "SPEC": {"NODE_NAME": "by", 'NBOR_NAME': "root", "NBOR_RELOP": ">"}
    },
    {
    "PATTERN": {"DEP": "pobj"},
    "SPEC": {"NODE_NAME": "val", 'NBOR_NAME': "by", "NBOR_RELOP": ">"}
    }
]

# Finnaly we want to extract dates
numpod_pattern = {
    "PATTERN": {"DEP": "nummod", "OP": "+"},
    "SPEC": {"NODE_NAME": "nums", 'NBOR_NAME': "val", "NBOR_RELOP": ">"}
}

date_pattern = [
    {
    "PATTERN": {"LEMMA": "found"},
    "SPEC": {"NODE_NAME": "root"}
    },
    {
    "PATTERN": {"DEP": "prep", "LEMMA": "in"},
    "SPEC": {"NODE_NAME": "in", 'NBOR_NAME': "root", "NBOR_RELOP": ">"}
    },
    {
    "PATTERN": {"DEP": "pobj"},
    "SPEC": {"NODE_NAME": "val", 'NBOR_NAME': "in", "NBOR_RELOP": ">"}
    }
]

for i, doc in enumerate(docs):
    print(f"doc{i}".center(20, "-"))
    matches = dependency_parser(founded_pattern, doc, additional_pattern=compound_pattern)
    print(matches)
    matches = dependency_parser(founder_pattern, doc, additional_pattern=compound_pattern)
    print(matches)
    matches = dependency_parser(date_pattern, doc, additional_pattern=numpod_pattern)
    print(matches)

--------doc0--------
[[founded, Corporation, Sony]]
[[founded, by, Ibuka, Masaru]]
[]
--------doc1--------
[[founded, Corporation, LG]]
[[founded, by, hwoi, Koo], [founded, by, hwoi, Koo], [founded, by, hwoi, In]]
[[founded, in, January, 5], [founded, in, January, 1947]]
--------doc2--------
[[founded, Corporation, Panasonic]]
[[founded, by, Matsushita, Kōnosuke]]
[[founded, in, March, 13], [founded, in, March, 1918]]


## 5. Add it as a component into the spaCy pipeline 

In [18]:
def get_org_info(doc):
    matches = dependency_parser(founded_pattern, doc, additional_pattern=compound_pattern)
    print(matches)
    matches = dependency_parser(founder_pattern, doc, additional_pattern=compound_pattern)
    print(matches)
    matches = dependency_parser(date_pattern, doc, additional_pattern=numpod_pattern)
    print(matches)
    return doc

if nlp.has_pipe("get_org_info"):
    nlp.remove_pipe("get_org_info")
    
nlp.add_pipe(get_org_info, after='ner')
print(nlp.pipe_names)

for i, text in enumerate((text0, text1, text2)):
    print(f"text{i}".center(20, "-"))
    doc = nlp(text)

['tagger', 'parser', 'ner', 'get_org_info']
-------text0--------
[[founded, Corporation, Sony]]
[[founded, by, Ibuka, Masaru]]
[]
-------text1--------
[[founded, Corporation, LG]]
[[founded, by, hwoi, Koo], [founded, by, hwoi, Koo], [founded, by, hwoi, In]]
[[founded, in, January, 5], [founded, in, January, 1947]]
-------text2--------
[[founded, Corporation, Panasonic]]
[[founded, by, Matsushita, Kōnosuke]]
[[founded, in, March, 13], [founded, in, March, 1918]]


# Part 2: Document clustering

## 1, 2, 3. Take a book, split to sentences, create spacy object 

In [47]:
import nltk
import re
import spacy
import random
from nltk.corpus import gutenberg 
from sklearn.cluster import KMeans
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
from urllib.request import urlopen
url = "https://www.gutenberg.org/files/158/158-0.txt"
raw = urlopen(url).read().decode('utf8')
len(raw)

raw.replace('\n', '')
sents = nltk.sent_tokenize(raw)
len(sents)

6027

In [49]:
nlp = spacy.load('en_core_web_sm')

In [50]:
spacy_sents = list(map(nlp, sents))

## 4. Use a clustering method of your choice.

In [51]:
n_clust = 5
model = KMeans(n_clusters=n_clust, random_state=0)
model.fit(list(map(lambda sent: sent.vector, spacy_sents))) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [52]:
from collections import defaultdict
def collect_clusters(model):
  clusters = defaultdict(list)
  for sent, cluster in zip(spacy_sents, model.labels_):
    clusters[cluster].append(sent)
  return clusters

In [53]:
def print_clusters(clusters):
  for cluster_key, cluster_sents in clusters.items():
    print('Cluster: ', cluster_key)
    for sent in random.choices(cluster_sents, k=5):
      print('>', sent)
    print()

In [54]:
random.seed(0)
print_clusters(collect_clusters(model))

Cluster:  0
> Such a debasement on his!
> Seven miles were travelled in expectation of enjoyment, and every body
had a burst of admiration on first arriving; but in the general amount
of the day there was deficiency.
> He was
not exactly what she had expected; less of the man of the world in some
of his notions, less of the spoiled child of fortune, therefore better
than she had expected.
> There is jealousy.
> I do not know her, even by sight.”

“I can answer for every thing of that nature, sir, because it will be
under Mrs. Weston's care.

Cluster:  2
> So very obliging of Mr. Frank Churchill!
> Elton, I suppose, has been the person to whom Miss Fairfax owes--”

“Yes, our good Mrs. Elton.
> A reasonable visit paid, Mr. Weston began to move.--“He must be going.
> Miss Woodhouse, I hope nothing may happen to prevent the ball.
> But this good old Mr. Woodhouse, I wish
you had heard his gallant speeches to me at dinner.

Cluster:  3
> From our relative situation, those
attentions were he